# Bounding H<sup>2</sup>MM

Let's get our obligitory imports in order, and we'll load the 3 detector data as well.

In [1]:
import os
import numpy as np
from matplotlib import pyplot as plt

import H2MM_C as hm

def load_txtdata(filename):
    """
    load the space-separated bursts file
    """
    color = list()
    times = list()
    with open(filename,'r') as f:
        for i, line in enumerate(f):
            if i % 2 == 0:
                times.append(np.array([int(x) for x in line.split()],dtype=np.int64))
            else:
                color.append(np.array([int(x) for x in line.split()],dtype=np.uint8))
    return color, times

color3, times3 = load_txtdata('sample_data_3det.txt')

## Built in Limits

Sometimes we may want to restrain what values are possible in the H<sup>2</sup>MM model, for instance to keep transition rates within reasonable values, or because you know something about how the emmission probability matrix (`obs`) should behave.

This is expected to happen most often (but not exclusively) when some experimental parameter is periodic, but not important to the data. For instance in $\mu$sALEX experiments, the laser alternation period causes donor and acceptor excitation photons, which arrive in separate streams to alternate (and thus transition) perfectly periodically, yet that has no bearing on transitions between our system. Thus transition rates close to the rate of laser alternation are likely to be artifacts, and thus we want to exclude them. For $\mu sALEX$ experiments, we find this still is not enough.

To define some bounds, we need to define the bounds, this is done using the `hm.h2mm_limits` object, which we pass into the `hm.EM_H2MM_C()` function through the keyword argument `bounds`, and we also need to supply a method string to another keyword argument, `bounds_func`.

#### Let's see an example

In [2]:
alt_period = 4000 # a fake alternation period
us_bounds = hm.h2mm_limits(max_trans = 1/(alt_period))

prior = np.array([1/4, 1/4, 1/4, 1/4])
trans = np.array([[1-3e-6, 1e-6, 1e-6, 1e-6],
                  [1e-6, 1-3e-6, 1e-6, 1e-6],
                  [1e-6, 1e-6, 1-3e-6, 1e-6],
                  [1e-6, 1e-6, 1e-6, 1-3e-6]])
obs = np.array([[0.4,0.4,0.2],
                [0.3,0.1,0.6],
                [0.2,0.4,0.4],
                [0.1,0.1,0.8]])

imodel_4s3d = hm.h2mm_model(prior, trans, obs)

us_opt_model4 = hm.EM_H2MM_C(imodel_4s3d, color3, times3, bounds_func='revert', bounds=us_bounds)
us_opt_model4

The model converged after 685 iterations


nstate: 4, ndet: 3, nphot: 436084, niter: 685, loglik: -408203.01780763146 converged state: 0x27
prior:
0.1974229271707277, 0.5611282263485258, 0.24144884648074666, 1.1225400688763535e-45
trans:
0.9999562426522093, 2.620891499277249e-05, 1.8189510111391653e-06, 1.5729481786860052e-05
7.049860033086557e-06, 0.9999698846938487, 6.991336733963458e-06, 1.6074109384330507e-05
1.2716821568198507e-06, 1.7388154493563684e-05, 0.9999781791356339, 3.161027715683239e-06
1.7300615959528795e-05, 0.00011452978544642954, 8.076561260936483e-06, 0.9998600930373331
obs:
0.8495291267451746, 0.07564775438957819, 0.0748231188652472
0.4716864635066343, 0.09134403951110474, 0.4369694969822609
0.1490998472367151, 0.31276921375661093, 0.538130939006674
0.15084733434554878, 0.07681326299829004, 0.7723394026561612

So, what did we just do? The `hm.h2mm_limits` object `us_bounds` prevents any value (off the diagonal) of the **transition probability** matrix (`.trans`) from ever being larger (i.e. faster transition rate) than `1/(4000)`. 

#### Bounds process

When you use a bounds method, each iteration goes through the following steps:
1. Calculate *loglikelihood* and new model
2. Check if the **model** converged
3. Analyze the **new model**, and correct if necessary
    1. Check if any values are smaller or larger than a pre-set minimum or maximum
    2. If values are out of bounds, apply correction, method defined by argument passed to `bounds_func`
4. Repeat optimization (back to step 1)

The inputs to `hm.h2mm_limits` are all keyword argumetns, and come in the form of `min/max_[array]` where `[array]` is `prior`, `trans` or `obs`, and specify the minimum and maximum values in the respective array.
Specifying as a float will set the value for all states, and thus the created `hm.h2mm_limits` object can be used for models with any model, while values can be specified as arrays, where each element sets the min/max of the value at that position in the given array of the model.

#### `bounds_func`

As mentioned in the above outline, the bounding process needs to choose how to correct the way in which a model value that is out of bound is corrected.
There are 3 options:

1. `minmax` shallowest correction, sets the value to its minimum or maximum
2. `revert` prefered method, sets the value to the value in the previous model
3. `revert_old` a more extreme form of `revert` which goes to the model before the last in the optimization, and sets the value to that.

### Using `hm.factory_h2mm_model()` with bounds

You will note in the previous example, we specified the `hm.h2mm_model` explicitly, instead of using `hm.factory_h2mm_model()`. 
This is because it is possible that the `hm.factory_h2mm_model()` could create an initial model that contains out of bounds values, which could result in odd behavior during optimization.

There is a way around this, you can give the `hm.h2mm_limits` object to `hm.factory_h2mm_model()` through the keyword argument `bounds`, and the function will automatically ensure the model is with bounds:

> See the full documentation to see full list of options for customizing the `hm.factory_h2mm_model()` function's output

In [3]:
us_bounds = hm.h2mm_limits(max_trans = 1/4000)
# make factory_h2mm_model make a model within bounds
imodel = hm.factory_h2mm_model(3,3, bounds=us_bounds)
us_model = hm.EM_H2MM_C(imodel, color3, times3, bounds=us_bounds, bounds_func='revert')

The model converged after 203 iterations


## Custom Bounds

Finally, it is possible to supply a custom bounding function to `bounds_func`.

> **Note**
>
> This feature was designed to allow the user to handle things/circumstances
> that the writers of H2MM_C had not anticipated.
> Therefore this example is very simple, and does not show a useful method.

This function must is called at the end of the optimization loop, after a given model's loglik has been 
calculated (and the standard H<sup>2</sup>MM next model for the next iteration produced).

This function takes the signature 
`bounds_func(new:h2mm_model, current:h2mm_model, old:h2mm_model, *bounds, **bounds_func)->h2mm_model|int|tuple[h2mm_model,int]`

`new`, `current` and `old` are the `h2mm_model`s of the current iteration.

1. `new` is the model suggested/produced by the current iteration
2. `current` is the model whose loglik was just computed
3. `old` is the model computed in the previous iteration

These are always supplied each iteration. `bounds` and `bounds_kwargs` come from the
identically named keyword arguments in `EM_H2MM_C`. `bounds` by default is `None`, which 
is internally converted to a 0-size (empty) `tuple`, likewise `bounds_kwargs` is by default `None`
and is internally converted into an empty dict.

The return value can either or both specify
1. The "bounded" `new` model
2. If the optimization has converged

If only the `new` model is specified, convergence will be determined like all other optimizaztions,
by the difference in loglik of `current` and `new`. 

**However,** if the bounds function returns a value specifying if the model has converged, then
`EM_H2MM_C` will **not** separately check if the optimization has converged.

> **Note**
> `max_iter` and `max_time` are enforced separetely from `bounds_func`. 

If specifying the converged state, this can be either a `bool` or 0, 1, 2.

As a `bool`, `True` indicates that the optimization has converged, and thus
can stop, the `old` model will be returned as the "optimal" model. `False`
will allow the optimization to proceed using the `new` model.

If specifying as `0` is equivalent to `False`, `1` to `True`, and `2` will return
the 'current' model as the optimal model.

If both the `new` model and converged state are specified, this must be done by returning a 2-tuple
of `(new, converged_state)`.

> **Warning**
> Make sure the model you return makes sense, otherwise the optimization will proceed unpredictably.
> Think of this as the “gloves off” approach, you might have a very powerful new method, or you might
> get something meaningless depending on how you code it. That’s your responsibility.

Below is a function that that re-implements the behavior of `"minmax"` but now the limits
normally specified with a `h2mm_limits` object supplied to `bounds` are replaced with kwargs:

In [4]:
def minmax_py(new, current, old, converged_min=1e-9, 
                  min_prior=None, max_prior=None,
                  min_trans=None, max_trans=None, 
                  min_obs=None, max_obs=None):
    # bounding of trans matrix
    if min_trans is not None or max_trans is not None:
        trans = new.trans
        idxs = np.arange(new.nstate)
        if isinstance(min_trans, float):
            trans[trans < min_trans*(~np.eye(new.nstate, dtype=np.bool_))] = min_trans
        elif isinstance(min_trans, np.ndarray):
            mask = trans < min_trans
            trans[mask] = min_trans[mask]
        if isinstance(max_trans, float):
            trans[trans > max_trans*(~np.eye(new.nstate, dtype=np.bool_))] = max_trans
        elif isinstance(max_trans, np.ndarray):
            mask = trans > max_trans
            trans[mask] = max_trans[mask]
        for i in range(trans.shape[0]):
            trans[i,i] = 1.0 - trans[i, idxs!=i].sum()
        new.trans = trans
    # bounding of obs matrix
    if min_obs is not None or max_obs is not None:
        obs = new.obs
        if min_obs is not None:
            minmask = obs < min_obs
            obs[minmask] = min_obs[minmask]
        else:
            minmask = np.zeros(obs.shape, dtype=np.bool_)
        if max_obs is not None:
            maxmask = obs > max_obs
            obs[maxmask] = max_obs[maxmask]
        else:
            maxmask = np.zeros(obs.shape, dtype=np.bool_)
        obsmask = minmask | maxmask
        for i in range(obs.shape[0]):
            obs[i,~obsmask[i,:]] += (1-obs[i,:].sum()) / (~obsmask).sum()
        new.obs = obs
    if min_prior is not None or max_prior is not None:
        prior = new.prior
        if min_prior is not None:
            minpmask = prior < min_prior
            prior[minpmask] = min_prior[minpmask]
        else:
            minpmask = np.zeros(new.nstate, base=np.bool_)
        if max_prior is not None:
            maxpmask = prior > max_prior
            prior[maxpmask] = max_prior[maxpmask]
        else:
            maxpmask = np.zeros(new.nstate, base=np.bool_)
        pmask = minpmask | maxpmask
        prior[~pmask] += (1-prior.sum()) / (~pmask).sum()
        new.prior = prior
    return new

In [5]:
prior = np.array([1/4, 1/4, 1/4, 1/4])
trans = np.array([[1-3e-6, 1e-6, 1e-6, 1e-6],
                  [1e-6, 1-3e-6, 1e-6, 1e-6],
                  [1e-6, 1e-6, 1-3e-6, 1e-6],
                  [1e-6, 1e-6, 1e-6, 1-3e-6]])
obs = np.array([[0.09,0.01,0.9],
                [0.3,0.1,0.6],
                [0.2,0.4,0.4],
                [0.1,0.1,0.8]])

imodel4s3d = hm.h2mm_model(prior, trans, obs)
us_opt_model4 = hm.EM_H2MM_C(imodel_4s3d, color3, times3, bounds_func=minmax_py, bounds_kwargs=dict(max_trans=1e-4))
us_opt_model4

The model converged after 357 iterations


nstate: 4, ndet: 3, nphot: 436084, niter: 357, loglik: -408204.56206646416 converged state: 0x27
prior:
0.20187287395552453, 0.5555113144684087, 0.2426158115760668, 1.1509971417548342e-19
trans:
0.9999561167094794, 2.578052243052947e-05, 1.8078370356740157e-06, 1.629493105452633e-05
6.756349830041571e-06, 0.9999721939851459, 6.949840784328318e-06, 1.4099824239669664e-05
1.2573173537528058e-06, 1.7601657363283022e-05, 0.9999780805061623, 3.0605191207252818e-06
1.9997368768734257e-05, 0.0001, 8.607871108432096e-06, 0.9998713947601229
obs:
0.8487004871938236, 0.07578053474456366, 0.07551897806161274
0.4703530361644828, 0.09123833982326242, 0.4384086240122547
0.14917488290484957, 0.3128863710402288, 0.5379387460549216
0.152440399778544, 0.07711927285636493, 0.7704403273650912

Now let's re-implement how convergence of the optimization is handled:

In [6]:
def limit_converged(new, current, old, conv_min):
    if current.loglik < old.loglik:
        return 1
    if (current.loglik - old.loglik) < conv_min:
        return 2
    return 0

In [7]:
us_opt_model4 = hm.EM_H2MM_C(imodel_4s3d, color3, times3, bounds_func=limit_converged, bounds=5e-8)
us_opt_model4

The model converged after 590 iterations


nstate: 4, ndet: 3, nphot: 436084, niter: 590, loglik: -408203.01780927135 converged state: 0x27
prior:
0.19742858374944944, 0.561121392455406, 0.24145002379514446, 5.663724687881962e-39
trans:
0.999956242650991, 2.6207641738997258e-05, 1.8189787919602536e-06, 1.5730728478121535e-05
7.049515187084346e-06, 0.9999698870139253, 6.991349840780732e-06, 1.6072121046822835e-05
1.2716786501567222e-06, 1.738831022553686e-05, 0.9999781790484651, 3.160962659245479e-06
1.730359295110415e-05, 0.00011451967406835532, 8.076758043081436e-06, 0.9998600999749374
obs:
0.8495279860407298, 0.07564793243468458, 0.07482408152458554
0.4716848692803994, 0.09134393961306514, 0.4369711911065356
0.14909992362596466, 0.3127691549445517, 0.5381309214294837
0.1508460096123576, 0.0768130082594132, 0.7723409821282292

Finally, bellow is an example that re-implements the min-max procedure and checking for convergence:

In [8]:
def minmax_conv_py(new, current, old, conv_min, **kwargs):
    return minmax_py(new, current, old, **kwargs), limit_converged(new, current, old, conv_min)

In [9]:
us_opt_model4 = hm.EM_H2MM_C(imodel_4s3d, color3, times3, bounds_func=minmax_conv_py, bounds=5e-8, bounds_kwargs=dict(max_trans=1e-4))
us_opt_model4

The model converged after 312 iterations


nstate: 4, ndet: 3, nphot: 436084, niter: 312, loglik: -408204.56206726446 converged state: 0x27
prior:
0.2018750928494704, 0.5555092492855975, 0.24261565786493222, 3.425647254746712e-17
trans:
0.9999561169583928, 2.5778882951706083e-05, 1.8080121398063688e-06, 1.629614651565672e-05
6.756074148398783e-06, 0.9999721942825246, 6.949971979473416e-06, 1.4099671347632825e-05
1.2573545270720623e-06, 1.760180263309594e-05, 0.9999780806157591, 3.06022708071752e-06
1.999900172562418e-05, 0.0001, 8.606627635930037e-06, 0.9998713943706384
obs:
0.8486998060312413, 0.07578060671871177, 0.07551958725004693
0.4703530221933338, 0.0912383127763227, 0.43840866503034354
0.14917480330653876, 0.312885940969217, 0.5379392557242443
0.15244034337688986, 0.07711913838505483, 0.7704405182380554